In [57]:
import pandas as pd
import json

In [58]:
from key_features import *

In [59]:
fm = pd.read_csv('data/fm_04092022.csv')

In [60]:
fm.head()

,UID,Name,Nat,Based,Club,Position,Preferred Foot,Right Foot,Left Foot,Height,...,Bal_20,Kic_20,Sta_20,Agi_20,Wor_20,Bra_20,Cmd_20,Fin_20,1v1_20,TRO_20
0,7458500,Lionel Messi,ARG,France (Ligue 1),PSG,"AM (RC), ST (C)",Left,Fairly Strong,Very Strong,169 cm,...,19,1,13,19,7,10,3,20,3,1
1,18004457,Kevin De Bruyne,BEL,England (Premier League),Manchester City,"M (RLC), AM (C)",Either,Very Strong,Strong,181 cm,...,14,4,17,13,15,14,2,16,2,1
2,735216,Cristiano Ronaldo,POR,England (Premier League),Man Utd,"AM (L), ST (C)",Either,Very Strong,Strong,187 cm,...,14,2,15,13,10,16,4,19,2,2
3,85139014,Kylian Mbappé,FRA,France (Ligue 1),PSG,"AM (RL), ST (C)",Right,Very Strong,Reasonable,178 cm,...,14,3,14,16,10,12,1,17,3,3
4,98028755,Mohamed Salah,EGY,England (Premier League),Liverpool,"AM (RL), ST (C)",Left,Weak,Very Strong,175 cm,...,17,1,14,16,14,11,3,18,2,3


In [61]:
{'key': GK['Sweeper Keeper (Defend)'][0], 'preferred': GK['Sweeper Keeper (Defend)'][1]}

KeyError: 0

In [ ]:
for key, value in GK.items():
    GK[key] = {'key': value[0], 'preferred':value[1]}

In [ ]:
for key, value in DEF.items():
    DEF[key] = {'key': value[0], 'preferred':value[1]}

In [ ]:
for key, value in MID.items():
    MID[key] = {'key': value[0], 'preferred':value[1]}

In [ ]:
for key, value in FW.items():
    FW[key] = {'key': value[0], 'preferred':value[1]}

In [ ]:
positions = {'GK': GK, 'DEF': DEF, 'MID': MID, 'FW': FW}

In [ ]:
for key, value in positions.items():
    for role, attributes in value.items():
        attributes['key'] = [x.strip() for x in attributes['key']]
        attributes['preferred'] = [x.strip() for x in attributes['preferred']]

In [ ]:
json.dump(positions,indent=2,fp=open('data/positions.json','w'))

In [ ]:
scores = pd.DataFrame()
scores['UID'] = fm['UID']
scores['Name'] = fm.Name
scores.head()

In [ ]:
all_attributes = []

for key, value in positions.items():
    for role, attributes in value.items():
        all_attributes.extend(attributes['key'])
        all_attributes.extend(attributes['preferred'])

In [ ]:
all_attributes = list(set(all_attributes))
len(all_attributes)

In [ ]:
NON_GK_ATRIBUTES = [x.strip() for x in NON_GK_ATRIBUTES]
NON_PLAYER_ATTRIBUTES = [x.strip() for x in NON_PLAYER_ATTRIBUTES]
NON_GK_ATRIBUTES

In [ ]:
for pos_name, roles in positions.items():
    for role_name, attributes in roles.items():
        keys_22 = attributes['key']
        preferred_22 = attributes['preferred']
        keys_21 = [x + '_21' for x in keys_22]
        keys_20 = [x + '_20' for x in keys_22]
        preferred_21 = [x + '_21' for x in preferred_22]
        preferred_20 = [x + '_20' for x in preferred_22]

        if pos_name == 'GK':
            others_22 = [x for x in all_attributes if x not in keys_22 and x not in preferred_22 and x not in NON_GK_ATRIBUTES]
        else:
            others_22 = [x for x in all_attributes if x not in keys_22 and x not in preferred_22 and x not in NON_PLAYER_ATTRIBUTES]

        others_21 = [x + '_21' for x in others_22]
        others_20 = [x + '_20' for x in others_22]
        scores[role_name] = (fm[keys_22].mean(axis=1) * 0.55 + fm[preferred_22].mean(axis=1) * 0.35+ fm[others_22].mean(axis=1)*0.1)*0.6\
                            +(fm[keys_21].mean(axis=1) * 0.55 + fm[preferred_21].mean(axis=1) * 0.35+ fm[others_22].mean(axis=1)*0.1)*0.3\
                            +(fm[keys_20].mean(axis=1) * 0.55 + fm[preferred_20].mean(axis=1) * 0.35+ fm[others_22].mean(axis=1)*0.1)*0.1

        print(f"{pos_name} {role_name} done")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler((0, 100))
scores.iloc[:,2:] = pd.DataFrame(scaler.fit_transform(scores.iloc[:,2:]), columns=scores.iloc[:,2:].columns)

In [ ]:
scores

In [ ]:
searched_ids = fm[(fm['Based'] == 'Turkey (Süper Lig)')]['UID'].values

In [ ]:
selected_role = 'Defensive Midfielder (Defend)'
scores.loc[scores['UID'].isin(searched_ids),['Name',selected_role]].sort_values(by=selected_role, ascending=False).head(10)

In [ ]:
#scores.to_csv('data/scores_13092022.csv', index=False)

***

In [ ]:
import numpy as np

In [ ]:
managers = pd.read_csv('data/fm22managers.csv')
managers.columns = [x.strip() for x in managers.columns]
managers = managers.applymap(lambda x: x.strip() if isinstance(x, str) else x)
managers = managers.applymap(lambda x: np.nan if x=='' else x)
managers = managers.dropna(subset=['Club'])
managers = managers.drop_duplicates(subset=['Club'], keep='first')

In [ ]:
miguel_crespo = fm[fm['Name'] == 'Miguel Crespo']

In [ ]:
miguel_crespo

In [ ]:
scores[scores['UID'] == miguel_crespo.UID.values[0]]

In [ ]:
managers[managers['Club'] == miguel_crespo.Club.values[0]]